In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""

Predict future albedo using empirical albedo model and 2015-2100 temperature data.

"""

# Import modules
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4
from scipy import stats
import glob
import pathlib
import os
import matplotlib.pyplot as plt

In [20]:
# Define path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

# Define savepath
sp = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/'

In [21]:
# Define summer albedo and temp data
ds1 = xr.open_dataset(path + 'temp_albedo_summer_climatologies.nc')

# Define CMIP6 data
cmip_files = sorted(glob.glob(path + 'cmip6-summer-resampled/*/*.nc'))

# Define empirical model
model = pd.read_csv(path + 'empirical_albedo_model.csv')

# Define cloud fraction
cloud_file = netCDF4.Dataset(path + 'modis_cloud_properties/cloud_fraction_2010.nc')
cloud_fraction = cloud_file.variables['cloud_fraction'][:]

# Define 2.5 K bins
bins = np.arange(-7.5, 20, 2.5)

# Define reference grids for the 2015-2020 period
clrsky_albedo = np.mean(ds1['albedo'][:,:,12:], axis=2)

# Convert clear-sky albedo to all-sky albedo
correction = cloud_fraction * 5.0
reference_albedo = clrsky_albedo + correction

In [ ]:
for j in range(len(cmip_files)):
    
    print('Processing %i out of %i' %(j+1, len(cmip_files)))
         
    # Read model
    cmip = xr.open_dataset(cmip_files[j])

    # Get some file names
    p = pathlib.Path(cmip_files[j])
    folder = p.parent.name
    savepath = sp + folder + '/' + p.name

    # Define reference grid for CMIP!!! 2015-2020 period
    reference_temp = np.mean(cmip['t2m'][:,:,:6], axis=2)
    reference_temp = reference_temp.where(reference_temp != -999)
    reference_temp = reference_temp.where(ds1['mask'] != 0)

    # Define empty array
    predicted_albedo = np.zeros((cmip['t2m'].shape[0], cmip['t2m'].shape[1]))

    for year in range(cmip['t2m'].shape[2]):

        # Normalize temperature to find bins
        summer_temp = cmip['t2m'][:,:,year].where(cmip['t2m'][:,:,year] != -999)
        normalized_temp = summer_temp - np.nanmean(reference_temp)

        # Find temperature difference
        temp_diff = summer_temp - reference_temp

        # Define empty array
        predicted_albedo_stack = np.zeros(temp_diff.shape)

        for i in range(len(bins) - 1):

            # Mask
            mask = (normalized_temp > bins[i]) & (normalized_temp < bins[i+1])

            # Stack
            predicted_albedo_stack = np.dstack((predicted_albedo_stack, reference_albedo + temp_diff.where(mask == 1) * model.iloc[i]['slope']))

        predicted_albedo_year = np.nansum(predicted_albedo_stack, axis=2)

        # Convert 0 to NaNs
        predicted_albedo_year[predicted_albedo_year == 0] = np.nan

        # Convert values < 0.3
        predicted_albedo_year[predicted_albedo_year < 30] = 30

        # Stack
        predicted_albedo = np.dstack((predicted_albedo, predicted_albedo_year))

    predicted_albedo = predicted_albedo[:,:,1:]

    # Save as NetCDF
    dataset = netCDF4.Dataset(savepath, 'w', format='NETCDF4_CLASSIC')
    print('Creating... %s' % savepath)
    dataset.Title = "Predicted albedo to 2100 based on summer 2 m air temperature"
    import time
    dataset.History = "Created " + time.ctime(time.time())
    dataset.Projection = "WGS 84"
    dataset.Reference = "Ryan, J. C., Smith. L. C., Cooley, S. W., and Pearson, B. (in review), Emerging importance of clouds for Greenland Ice Sheet energy balance and meltwater production."
    dataset.Contact = "jryan4@uoregon.edu"

    # Create new dimensions
    lat_dim = dataset.createDimension('y', predicted_albedo.shape[0])
    lon_dim = dataset.createDimension('x', predicted_albedo.shape[1])
    data_dim = dataset.createDimension('z', predicted_albedo.shape[2])

    # Define variable types
    Y = dataset.createVariable('latitude', np.float32, ('y', 'x'))
    X = dataset.createVariable('longitude', np.float32, ('y', 'x'))

    # Define units
    Y.units = "degrees"
    X.units = "degrees"

    # Create the actual 3D variable
    albedo_nc = dataset.createVariable('albedo', np.float32, ('y','x','z'))

    # Write data to layers
    Y[:] = ds1['latitude']
    X[:] = ds1['longitude']
    albedo_nc[:] = predicted_albedo

    print('Writing data to %s' % savepath)

    # Close dataset
    dataset.close()

Processing 1 out of 78
Creating... /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_ACCESS-CM2_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Writing data to /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_ACCESS-CM2_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Processing 2 out of 78
Creating... /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_ACCESS-ESM1-5_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Writing data to /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_ACCESS-ESM1-5_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Processing 3 out of 78
Creating... /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_CESM2-WACCM_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Writing data to /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-albedo/SSP126/CMIP6_CESM2-WACCM_Amon_ssp126_r1i1p1f1_tas_2015-2